In [2]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from numpy.linalg import norm, solve

import pinocchio

In [89]:
model = pinocchio.buildModelFromUrdf("../panda-model/panda_arm.urdf")
data  = model.createData()

JOINT_ID = 7
oMdes = pinocchio.SE3(np.eye(3), np.array([0.5, 0., 0.5]))

q      = pinocchio.neutral(model)
eps    = 1e-4
IT_MAX = 1000
DT     = 1e-1
damp   = 1e-12

i=0
while True:
    pinocchio.forwardKinematics(model,data,q)
    dMi = oMdes.actInv(data.oMi[JOINT_ID])
    err = pinocchio.log(dMi).vector
    if norm(err) < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pinocchio.computeJointJacobian(model,data,q,JOINT_ID)
    v = - J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pinocchio.integrate(model,q,v*DT)

    i += 1

if success:
    print("Convergence achieved!")
else:
    print("\nWarning: the iterative algorithm has not reached convergence to the desired precision")

print('\nresult: %s' % q.flatten().tolist())
print('\nfinal error: %s' % err.T)

qTarget = np.array([0.9066, 1.16943, 1.44188, -1.31434, 1.20663, 1.786, 0.473774])
qTarget

Convergence achieved!

result: [0.9061736122419272, 1.172743450007835, 1.4385310564139155, 1.3142170663092332, 1.2105764728414783, 1.7884386101242125, 0.47454134904666545]

final error: [-1.99092757e-05 -2.47038394e-05  3.52305088e-06 -8.87726879e-05
  1.09818309e-05 -1.23876571e-05]


array([ 0.9066  ,  1.16943 ,  1.44188 , -1.31434 ,  1.20663 ,  1.786   ,
        0.473774])

In [114]:
data = np.loadtxt("optimal_solution.txt").T

time = data[0]
position = data[1:8]
velocity = data[8:15]
acceleration = data[15:22]

position_limit = np.array([[-2.8973, -1.7628, -2.8973, -3.0718, -2.8973, -0.0175, -2.8973], 
                          [2.8973, 1.7628, 2.8973, -0.0698, 2.8973, 3.7525, 2.8973]])

velocity_limit = np.array([2.1750, 2.1750, 2.1750, 2.1750, 2.6100, 2.6100, 2.6100])

acceleration_limit = np.array([15.0, 7.5, 10.0, 12.5, 15.0, 20.0, 20.0])


fig = make_subplots(rows=7, cols=3)

for i in range(7):

     # Plot position and their limit
    fig.add_trace(
    go.Scatter(x=time, y=position[i, :]),
    row=i+1, col=1 )

    fig.add_hline(y=qTarget[i],
    row=i+1, col=1, line_dash="dash", line_width=0.5)

    fig.add_hline(y=position_limit[0, i],
    row=i+1, col=1, line_dash="dash", line_width=1)

    fig.add_hline(y=position_limit[1, i],
    row=i+1, col=1, line_dash="dash", line_width=1)

    # Plot velocity and their limit
    fig.add_trace(
    go.Scatter(x=time, y=velocity[i, :]),
    row=i+1, col=2 )

    fig.add_hline(y=velocity_limit[i],
    row=i+1, col=2, line_dash="dash", line_width=1)

    fig.add_hline(y=-velocity_limit[i],
    row=i+1, col=2, line_dash="dash", line_width=1)

    # Plot acceleration and their limit
    fig.add_trace(
    go.Scatter(x=time, y=acceleration[i, :]),
    row=i+1, col=3 )

    fig.add_hline(y=acceleration_limit[i],
    row=i+1, col=3, line_dash="dash", line_width=1)

    fig.add_hline(y=-acceleration_limit[i],
    row=i+1, col=3, line_dash="dash", line_width=1)
    
fig.update_layout(height=2000, width=1000, title_text="PolyMPC collocation_fix_jw branch, poly=10, segment=1")
fig.update(layout_showlegend=False)
fig.show()